# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle
from keras.utils import to_categorical

Using TensorFlow backend.


# Load processed dataset
The data we load here are already shuffled and normalized.

In [2]:
data_directory = 'dataset/3. normalized/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

x_train: (966, 26)
y_train: (966, 2)
x_test: (242, 26)
y_test: (242, 2)


In [3]:
pd.DataFrame(x_train).head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.226135,0.165461,0.218373,0.136944,0.218364,0.326976,0.467346,0.250757,0.120083,0.321016,...,0.100836,0.338785,0.407917,0.155705,0.151108,0.181358,0.077172,0.484244,0.416667,0.675275
1,0.079400,0.089118,0.061998,0.030564,0.062079,0.089922,0.191393,0.043280,0.024083,0.080073,...,0.071548,0.018426,0.228516,0.074497,0.071189,0.362160,0.183293,0.296468,0.083333,0.077304
2,0.103458,0.103848,0.100653,0.065950,0.100690,0.158392,0.283503,0.082511,0.046162,0.196872,...,0.033205,0.257567,0.234512,0.075839,0.075218,0.310095,0.049176,0.000000,0.000000,0.000000
3,0.146945,0.220966,0.091491,0.064243,0.091571,0.334515,0.488109,0.191163,0.105050,0.330666,...,0.032132,0.115608,0.119201,0.020805,0.020148,0.553460,0.109024,0.291681,0.000000,0.000000
4,0.130953,0.136021,0.060241,0.029970,0.060280,0.179025,0.251038,0.107355,0.057036,0.181617,...,0.076168,0.191104,0.285241,0.037584,0.035594,0.328655,0.115067,0.408358,0.083333,0.482370


In [4]:
pd.DataFrame(x_test).head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.092726,0.130350,0.048820,0.025074,0.048818,0.206552,0.278973,0.123197,0.069596,0.157321,...,0.030981,0.126401,0.125133,0.033557,0.032908,0.496924,0.083581,0.357097,0.000000,0.000000
1,0.017255,0.026758,0.015688,0.010905,0.015687,0.108347,0.153643,0.081450,0.033064,0.096120,...,0.003376,0.186343,0.123078,0.209396,0.208865,0.466729,0.007642,0.000000,0.000000,0.000000
2,0.178509,0.286687,0.142696,0.088427,0.142732,0.279412,0.436769,0.188018,0.079033,0.236513,...,0.027032,0.099181,0.093156,0.045638,0.042982,0.606600,0.101034,0.203158,0.166667,0.266331
3,0.156274,0.177721,0.077686,0.058234,0.077682,0.160971,0.343903,0.068792,0.033838,0.099012,...,0.083206,0.102144,0.217839,0.032215,0.030893,0.377669,0.122733,0.614431,0.000000,0.000000
4,0.240163,0.460682,0.150351,0.120549,0.150387,0.305724,0.386183,0.190613,0.076670,0.253248,...,0.025370,0.029456,0.043714,0.011409,0.010745,0.765075,0.141341,0.567164,0.000000,0.000000


# Tunable parameters

In [5]:
epochs = 5000
batch_size = 20
learning_rate = 0.0001
decay = 0.0001

# Load autoencoder #1

In [6]:
auto1_dir = "results/results-autoencoder-01-02/model/2018-03-05-18-34-05/"
auto1_model = load_model(auto1_dir + 'weights.hdf5')
x_train_auto1 = auto1_model.predict(x_train)
x_test_auto1 = auto1_model.predict(x_test)

# Autoencoder #2

In [7]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/results-autoencoder-02-02/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"
def create_model(layers, dropout):
    global date
    x = Input(shape=(x_train_auto1.shape[1],))

    y = Dense(units = layers['layer_1'], activation='relu')(x)
    y = Dropout(dropout)(y)

    if (layers['layer_2'] > 0):
        y = Dense(units = layers['layer_2'], activation='relu')(y)
        y = Dropout(dropout)(y)
    
    y = Dense(units = x_train_auto1.shape[1], activation='sigmoid')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Layer 1': layers['layer_1'],
        'Layer 2': layers['layer_2'],
        'Dropout': dropout,
        'Learning Rate': learning_rate,
        'Decay': decay,
        'Epochs': epochs,
        'Batch size': batch_size
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [8]:
def get_callbacks():
    date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
    csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
    model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
    callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
    return callbacks

# Grid search

In [9]:
model = create_model({'layer_1': 10, 'layer_2': 0}, 0.4)
model.fit(x=x_train_auto1, y=x_train_auto1, batch_size=batch_size, epochs=epochs, validation_data=(x_test_auto1, x_test_auto1), callbacks=get_callbacks())

Train on 966 samples, validate on 242 samples
Epoch 1/5000
966/966 [==============================] - 0s - loss: 24.0568 - acc: 0.0280 - val_loss: 23.4633 - val_acc: 0.0000e+00
Epoch 2/5000
966/966 [==============================] - 0s - loss: 24.0239 - acc: 0.0311 - val_loss: 23.4299 - val_acc: 0.0000e+00
Epoch 3/5000
966/966 [==============================] - 0s - loss: 23.9814 - acc: 0.0135 - val_loss: 23.3989 - val_acc: 0.0124
Epoch 4/5000
966/966 [==============================] - 0s - loss: 23.9613 - acc: 0.0207 - val_loss: 23.3701 - val_acc: 0.0289
Epoch 5/5000
966/966 [==============================] - 0s - loss: 23.9379 - acc: 0.0528 - val_loss: 23.3416 - val_acc: 0.1157
Epoch 6/5000
966/966 [==============================] - 0s - loss: 23.9060 - acc: 0.0911 - val_loss: 23.3130 - val_acc: 0.2190
Epoch 7/5000
966/966 [==============================] - 0s - loss: 23.8846 - acc: 0.1687 - val_loss: 23.2817 - val_acc: 0.3264
Epoch 8/5000
966/966 [==============================] - 0

966/966 [==============================] - 0s - loss: 23.1144 - acc: 0.6449 - val_loss: 22.4032 - val_acc: 0.9050
Epoch 65/5000
966/966 [==============================] - 0s - loss: 23.1047 - acc: 0.6687 - val_loss: 22.3985 - val_acc: 0.9050
Epoch 66/5000
966/966 [==============================] - 0s - loss: 23.1075 - acc: 0.6429 - val_loss: 22.3928 - val_acc: 0.9050
Epoch 67/5000
966/966 [==============================] - 0s - loss: 23.0848 - acc: 0.6770 - val_loss: 22.3884 - val_acc: 0.9050
Epoch 68/5000
966/966 [==============================] - 0s - loss: 23.1026 - acc: 0.6625 - val_loss: 22.3832 - val_acc: 0.9050
Epoch 69/5000
966/966 [==============================] - 0s - loss: 23.0805 - acc: 0.6822 - val_loss: 22.3787 - val_acc: 0.9050
Epoch 70/5000
966/966 [==============================] - 0s - loss: 23.0859 - acc: 0.6812 - val_loss: 22.3740 - val_acc: 0.9050
Epoch 71/5000
966/966 [==============================] - ETA: 0s - loss: 23.3297 - acc: 0.69 - 0s - loss: 23.0806 - ac

966/966 [==============================] - 0s - loss: 22.8793 - acc: 0.8602 - val_loss: 22.1622 - val_acc: 0.9050
Epoch 128/5000
966/966 [==============================] - 0s - loss: 22.8681 - acc: 0.8737 - val_loss: 22.1588 - val_acc: 0.9050
Epoch 129/5000
966/966 [==============================] - 0s - loss: 22.8799 - acc: 0.8727 - val_loss: 22.1564 - val_acc: 0.9050
Epoch 130/5000
966/966 [==============================] - 0s - loss: 22.8675 - acc: 0.8737 - val_loss: 22.1534 - val_acc: 0.9050
Epoch 131/5000
966/966 [==============================] - 0s - loss: 22.8659 - acc: 0.8758 - val_loss: 22.1504 - val_acc: 0.9050
Epoch 132/5000
966/966 [==============================] - 0s - loss: 22.8675 - acc: 0.8561 - val_loss: 22.1478 - val_acc: 0.9050
Epoch 133/5000
966/966 [==============================] - 0s - loss: 22.8563 - acc: 0.8820 - val_loss: 22.1451 - val_acc: 0.9050
Epoch 134/5000
966/966 [==============================] - 0s - loss: 22.8550 - acc: 0.8872 - val_loss: 22.1414 -

966/966 [==============================] - 0s - loss: 22.7406 - acc: 0.9037 - val_loss: 22.0066 - val_acc: 0.9050
Epoch 191/5000
966/966 [==============================] - 0s - loss: 22.7242 - acc: 0.8913 - val_loss: 22.0045 - val_acc: 0.9050
Epoch 192/5000
966/966 [==============================] - 0s - loss: 22.7288 - acc: 0.8913 - val_loss: 22.0024 - val_acc: 0.9050
Epoch 193/5000
966/966 [==============================] - 0s - loss: 22.7245 - acc: 0.8892 - val_loss: 22.0006 - val_acc: 0.9050
Epoch 194/5000
966/966 [==============================] - 0s - loss: 22.7255 - acc: 0.8986 - val_loss: 21.9987 - val_acc: 0.9050
Epoch 195/5000
966/966 [==============================] - 0s - loss: 22.7234 - acc: 0.8892 - val_loss: 21.9967 - val_acc: 0.9050
Epoch 196/5000
966/966 [==============================] - 0s - loss: 22.7152 - acc: 0.9006 - val_loss: 21.9943 - val_acc: 0.9050
Epoch 197/5000
966/966 [==============================] - 0s - loss: 22.7110 - acc: 0.9017 - val_loss: 21.9936 -

966/966 [==============================] - 0s - loss: 22.6590 - acc: 0.9006 - val_loss: 21.9248 - val_acc: 0.9050
Epoch 254/5000
966/966 [==============================] - 0s - loss: 22.6552 - acc: 0.8965 - val_loss: 21.9243 - val_acc: 0.9050
Epoch 255/5000
966/966 [==============================] - 0s - loss: 22.6503 - acc: 0.9006 - val_loss: 21.9235 - val_acc: 0.9050
Epoch 256/5000
966/966 [==============================] - 0s - loss: 22.6678 - acc: 0.8986 - val_loss: 21.9226 - val_acc: 0.9050
Epoch 257/5000
966/966 [==============================] - 0s - loss: 22.6598 - acc: 0.9110 - val_loss: 21.9217 - val_acc: 0.9050
Epoch 258/5000
966/966 [==============================] - 0s - loss: 22.6405 - acc: 0.9048 - val_loss: 21.9212 - val_acc: 0.9050
Epoch 259/5000
966/966 [==============================] - 0s - loss: 22.6539 - acc: 0.9017 - val_loss: 21.9197 - val_acc: 0.9050
Epoch 260/5000
966/966 [==============================] - 0s - loss: 22.6409 - acc: 0.9006 - val_loss: 21.9190 -

...

966/966 [==============================] - 0s - loss: 22.4686 - acc: 0.9431 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4723/5000
966/966 [==============================] - 0s - loss: 22.4663 - acc: 0.9389 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4724/5000
966/966 [==============================] - 0s - loss: 22.4744 - acc: 0.9306 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4725/5000
966/966 [==============================] - 0s - loss: 22.4802 - acc: 0.9400 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4726/5000
966/966 [==============================] - 0s - loss: 22.4746 - acc: 0.9327 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4727/5000
966/966 [==============================] - 0s - loss: 22.4726 - acc: 0.9462 - val_loss: 21.7636 - val_acc: 0.9174
Epoch 4728/5000
966/966 [==============================] - 0s - loss: 22.4747 - acc: 0.9420 - val_loss: 21.7636 - val_acc: 0.9174
Epoch 4729/5000
966/966 [==============================] - 0s - loss: 22.4734 - acc: 0.9379 - val_loss: 21

966/966 [==============================] - 0s - loss: 22.4669 - acc: 0.9358 - val_loss: 21.7639 - val_acc: 0.9174
Epoch 4786/5000
966/966 [==============================] - 0s - loss: 22.4766 - acc: 0.9337 - val_loss: 21.7639 - val_acc: 0.9174
Epoch 4787/5000
966/966 [==============================] - 0s - loss: 22.4790 - acc: 0.9431 - val_loss: 21.7639 - val_acc: 0.9174
Epoch 4788/5000
966/966 [==============================] - 0s - loss: 22.4670 - acc: 0.9379 - val_loss: 21.7639 - val_acc: 0.9174
Epoch 4789/5000
966/966 [==============================] - 0s - loss: 22.4640 - acc: 0.9451 - val_loss: 21.7640 - val_acc: 0.9174
Epoch 4790/5000
966/966 [==============================] - 0s - loss: 22.4787 - acc: 0.9358 - val_loss: 21.7640 - val_acc: 0.9174.
Epoch 4791/5000
966/966 [==============================] - 0s - loss: 22.4697 - acc: 0.9420 - val_loss: 21.7640 - val_acc: 0.9174
Epoch 4792/5000
966/966 [==============================] - 0s - loss: 22.4814 - acc: 0.9462 - val_loss: 2

966/966 [==============================] - 0s - loss: 22.4712 - acc: 0.9358 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4849/5000
966/966 [==============================] - 0s - loss: 22.4700 - acc: 0.9348 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4850/5000
966/966 [==============================] - 0s - loss: 22.4718 - acc: 0.9389 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4851/5000
966/966 [==============================] - 0s - loss: 22.4801 - acc: 0.9389 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4852/5000
966/966 [==============================] - 0s - loss: 22.4697 - acc: 0.9400 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4853/5000
966/966 [==============================] - 0s - loss: 22.5015 - acc: 0.9255 - val_loss: 21.7638 - val_acc: 0.9174
Epoch 4854/5000
966/966 [==============================] - 0s - loss: 22.4753 - acc: 0.9369 - val_loss: 21.7638 - val_acc: 0.9174
Epoch 4855/5000
966/966 [==============================] - 0s - loss: 22.4717 - acc: 0.9358 - val_loss: 21

966/966 [==============================] - 0s - loss: 22.4639 - acc: 0.9400 - val_loss: 21.7638 - val_acc: 0.9174
Epoch 4912/5000
966/966 [==============================] - 0s - loss: 22.4773 - acc: 0.9420 - val_loss: 21.7638 - val_acc: 0.9174
Epoch 4913/5000
966/966 [==============================] - 0s - loss: 22.4678 - acc: 0.9296 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4914/5000
966/966 [==============================] - 0s - loss: 22.4699 - acc: 0.9441 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4915/5000
966/966 [==============================] - 0s - loss: 22.4727 - acc: 0.9389 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4916/5000
966/966 [==============================] - 0s - loss: 22.4705 - acc: 0.9389 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4917/5000
966/966 [==============================] - 0s - loss: 22.4651 - acc: 0.9348 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4918/5000
966/966 [==============================] - 0s - loss: 22.4806 - acc: 0.9379 - val_loss: 21

966/966 [==============================] - 0s - loss: 22.4749 - acc: 0.9420 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4975/5000
966/966 [==============================] - 0s - loss: 22.4747 - acc: 0.9389 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4976/5000
966/966 [==============================] - 0s - loss: 22.4652 - acc: 0.9410 - val_loss: 21.7637 - val_acc: 0.9174
Epoch 4977/5000
966/966 [==============================] - 0s - loss: 22.4742 - acc: 0.9369 - val_loss: 21.7636 - val_acc: 0.9174
Epoch 4978/5000
966/966 [==============================] - 0s - loss: 22.4810 - acc: 0.9420 - val_loss: 21.7636 - val_acc: 0.9174
Epoch 4979/5000
966/966 [==============================] - 0s - loss: 22.4715 - acc: 0.9296 - val_loss: 21.7636 - val_acc: 0.9174
Epoch 4980/5000
966/966 [==============================] - 0s - loss: 22.4742 - acc: 0.9379 - val_loss: 21.7636 - val_acc: 0.9174
Epoch 4981/5000
966/966 [==============================] - 0s - loss: 22.4736 - acc: 0.9420 - val_loss: 21